# Energy Storage Dispatch Algorithm
In this notebook, we'll lay out the algorithm for our **energy storage dispatch** in commercial buildings.

**Key variables:**
* Load Profile
* Power Prices
* Battery Characteristics

**Key outputs:**
* Price Savings

From which we'll derive -
* Dispatch Characteristics
* Modified Load Shape



In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Read Load Forecast
df_load = pd.read_csv(r'C:\Users\Niel Patel\Documents\GitHub\commercial_storage_analysis\data\Load_Price.csv')
df_load["Power Price ($/kWh)"] = df_load["Power Price ($/kWh)"].astype(float)
df_load["Battery_Max_Capacity"]= 1000
df_load["Battery_Current_Charge"]=0
df_load["Battery_Load"] = 0
df_load["Facility_Net_Load"] = df_load["Facility Load Data, pre-storage/renewables (kW)"]+df_load["Battery_Load"]
df_load["Pre-storage_Price"] = df_load["Facility Load Data, pre-storage/renewables (kW)"]*df_load["Power Price ($/kWh)"]*.25
df_load["Post-storage_Price"] = df_load["Facility_Net_Load"]*df_load["Power Price ($/kWh)"]*.25
df_load.head()




,Timestamp,"Facility Load Data, pre-storage/renewables (kW)",Power Price ($/kWh),Battery_Max_Capacity,Battery_Current_Charge,Battery_Load,Facility_Net_Load,Pre-storage_Price,Post-storage_Price
0,2016-07-29 00:15:00,2064,0.12,1000,0,0,2064,61.92,61.92
1,2016-07-29 00:30:00,2352,0.06,1000,0,0,2352,35.28,35.28
2,2016-07-29 00:45:00,2624,0.05,1000,0,0,2624,32.80,32.80
3,2016-07-29 01:00:00,2512,0.15,1000,0,0,2512,94.20,94.20
4,2016-07-29 01:15:00,2096,0.15,1000,0,0,2096,78.60,78.60


# Combining Data and Dispatch
In this section we'll be dipatching the battery to minimize the cost to the building.

**Battery Decision Options**
Charge
Discharge

**Logic**
Minimize Load(n) x PowerPrice(n) + Load(n+1) x PowerPrice(n+1) + Load (n+2) x PowerPrice(n+2)

Such that
Battery Charge <= Maximum Battery Capacity
Battery Charge >= 0


In [150]:
count = 0
last_pp = 0.10
last_charge = 0
for index, row in df_load.iterrows():
    now_pp = row["Power Price ($/kWh)"]
    
    #Loop for charging/discharging battery
    
    if now_pp < last_pp:
        
        #charge
        
        if last_charge >= 0 and last_charge < 1000:
            df_load.at[count,"Battery_Load"]= 1000 - last_charge
        df_load.at[count,"Battery_Current_Charge"] = 1000
        last_charge = df_load.at[count,"Battery_Current_Charge"]

    else:
        
        #discharge
        
        if last_charge > 0 and last_charge <= 1000:
            df_load.at[count,"Battery_Load"]= -last_charge
        df_load.at[count,"Battery_Current_Charge"] = 0
        last_charge = df_load.at[count,"Battery_Current_Charge"]
        
    last_pp = row["Power Price ($/kWh)"]
    count += 1
    
#Calculating Net Load
    
df_load["Facility_Net_Load"] = df_load["Facility Load Data, pre-storage/renewables (kW)"]+df_load["Battery_Load"]

#Calculating Net Price
df_load["Post-storage_Price"] = df_load["Facility_Net_Load"]*df_load["Power Price ($/kWh)"]*.25

Total_Price_Prestorage = df_load["Pre-storage_Price"].sum()
Total_Load_Prestorage = df_load["Facility Load Data, pre-storage/renewables (kW)"].sum()
print('The total pre-storage load is', Total_Load_Prestorage,'and total pre-storage price is',Total_Price_Prestorage)

Total_Price_Poststorage = df_load["Post-storage_Price"].sum()
Total_Load_Poststorage = df_load["Facility_Net_Load"].sum()
print('The total post-storage load is', Total_Load_Poststorage,'and total post-storage price is',Total_Price_Poststorage)

df_load


The total pre-storage load is 257544 and total pre-storage price is 6334.2699999999995
The total post-storage load is 257544 and total post-storage price is 6219.2699999999995


,Timestamp,"Facility Load Data, pre-storage/renewables (kW)",Power Price ($/kWh),Battery_Max_Capacity,Battery_Current_Charge,Battery_Load,Facility_Net_Load,Pre-storage_Price,Post-storage_Price
0,2016-07-29 00:15:00,2064,0.12,1000,0,0,2064,61.92,61.92
1,2016-07-29 00:30:00,2352,0.06,1000,1000,1000,3352,35.28,50.28
2,2016-07-29 00:45:00,2624,0.05,1000,1000,0,2624,32.80,32.80
3,2016-07-29 01:00:00,2512,0.15,1000,0,-1000,1512,94.20,56.70
4,2016-07-29 01:15:00,2096,0.15,1000,0,0,2096,78.60,78.60
5,2016-07-29 01:30:00,1920,0.14,1000,1000,1000,2920,67.20,102.20
6,2016-07-29 01:45:00,1888,0.06,1000,1000,0,1888,28.32,28.32
7,2016-07-29 02:00:00,2064,0.07,1000,0,-1000,1064,36.12,18.62
8,2016-07-29 02:15:00,2192,0.13,1000,0,0,2192,71.24,71.24
9,2016-07-29 02:30:00,2288,0.13,1000,0,0,2288,74.36,74.36
